In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from io import BytesIO
import cv2
import bson
from skimage.data import imread
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [17]:
def get_the_data(path):
    data = bson.decode_file_iter(open(path, 'rb'))
    images=[]
    category=[]
    for c, d in enumerate(data):
        product_id = d['_id']
        category_id = d['category_id'] # This won't be in Test data
        #prod_to_category[product_id] = category_id
        for e, pic in enumerate(d['imgs']):
            category.append(category_id)
            picture = imread(BytesIO(pic['picture']))
            #picture=pic['picture']
            images.append(picture)
            #break
        if(len(set(category))==500):
            break
    return category, images

In [18]:
product_category_train,image_train=get_the_data('E:\\kaggle\\train.bson')

In [19]:
data_len = len(product_category_train)

In [20]:
final=np.array(image_train)

In [21]:
def img2feat(im):
    return np.float32(im) / 255

final_train=img2feat(final)

In [22]:
y, rev_labels = pd.factorize(product_category_train)

In [23]:
len(y)

4696

In [24]:
img_pixel_val_flat = final_train.flatten().reshape(data_len, 97200)

In [25]:
np.unique(y, return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [26]:
from sklearn.utils import shuffle

im_train,lab_train=shuffle(final_train,y)
test_im=im_train[4600:]
test_lab=lab_train[4600:]
image_train=im_train[:4600]
label_train=lab_train[:4600]

In [27]:
from keras.layers import Conv2D, MaxPooling2D,Dropout,Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras import regularizers

In [28]:
model=Sequential()

model.add(Conv2D(16,3,activation='relu',input_shape=(180,180,3)))
model.add(Conv2D(16,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(2))
model.add(Dropout(0.2))

model.add(Conv2D(16,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(16,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(2))
model.add(Dropout(0.2))

model.add(Conv2D(32,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(32,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(2))
model.add(Dropout(0.2))

model.add(Conv2D(32,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(32,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(2))
model.add(Dropout(0.2))


model.add(Conv2D(64,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(64,3,activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(2))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels),activation='softmax'))

In [29]:
from keras import optimizers
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [30]:
model.fit(image_train,label_train,validation_split=0.2,epochs=15,batch_size=50)

Train on 3680 samples, validate on 920 samples
Epoch 1/15
3680/3680 [==============================] - 502s 137ms/step - loss: 20.3420 - acc: 0.0062 - val_loss: 20.3094 - val_acc: 0.3402
Epoch 2/15
3680/3680 [==============================] - 765s 208ms/step - loss: 20.2816 - acc: 0.2649 - val_loss: 20.2511 - val_acc: 0.3402
Epoch 3/15
3680/3680 [==============================] - 491s 133ms/step - loss: 20.2212 - acc: 0.3443 - val_loss: 20.1933 - val_acc: 0.3402
Epoch 4/15
3680/3680 [==============================] - 487s 132ms/step - loss: 20.1603 - acc: 0.3446 - val_loss: 20.1352 - val_acc: 0.3402
Epoch 5/15
3680/3680 [==============================] - 499s 136ms/step - loss: 20.0995 - acc: 0.3446 - val_loss: 20.0763 - val_acc: 0.3402
Epoch 6/15
3680/3680 [==============================] - 829s 225ms/step - loss: 20.0369 - acc: 0.3446 - val_loss: 20.0163 - val_acc: 0.3402
Epoch 7/15
3680/3680 [==============================] - 594s 161ms/step - loss: 19.9718 - acc: 0.3446 - val_loss:

In [31]:
pred=model.predict(test_im)

In [32]:
acc=[]
for i in pred:
    acc.append(np.argmax(i))

In [33]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(acc,test_lab)

In [34]:
accuracy

0.40625

In [36]:
from sklearn.metrics import recall_score
recall=recall_score(acc,test_lab, average='weighted')

C:\Users\saish\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [37]:
recall

0.40625

In [38]:
from sklearn.metrics import precision_score
pre=precision_score(acc,test_lab, average='weighted')

In [39]:
pre

1.0